In [13]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import logging
import DataProcessing

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Flatten, TimeDistributed, RepeatVector, Permute, Multiply, Lambda,Bidirectional
from keras import backend as K

dataProcess = DataProcessing.ETL()
df = dataProcess.Load_Clean_Data('./historical_data/Binance_ETHUSDT_d.csv','1d')

cols= ['ti_stoch_kd','ti_MACD']
cols_to_normalize = ['open','high', 'low', 'close', 'base_volume'] # removed num_trades
df = dataProcess.Add_TI_Data(df, cols=cols)
df= dataProcess.Add_Label(df,cols_to_normalize=cols_to_normalize)
#important: reverse the order and Reset the index to start from 0
df= df[::-1]
df.reset_index(drop=True, inplace=True)

#removed time ('unix_start_datetime')
x_cols = cols_to_normalize+cols
labels = ['result_A','result_B']
tensor_temp= df[x_cols+labels]
sequence_num=30# also called timesteps
tensor = dataProcess.create_tensor_with_sequence(tensor_temp,labels,sequence_num)
#note we have 2 labels, one for daily(B) another for shortterm (A) => shorterm>daily
#y1= df['result_A']
X= tensor.loc[:,~tensor.columns.isin(labels)]
y2= tensor['result_B']
y = y2.replace({'up': 1, 'down': -1, 'flat':0})
print(X.shape)
# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
# further split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# print the shape of each set
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)

#Important: reshape it into a 3D tensor with dimensions (number of samples, sequence length, number of features)
#to process with LSTM
X_train = X_train.reshape(X_train.shape[0],sequence_num,len(x_cols))
X_test = X_test.reshape(X_test.shape[0],sequence_num,len(x_cols))
X_val = X_val.reshape(X_val.shape[0],sequence_num,len(x_cols))



# seq_model = Sequential()
# # Add LSTM layer with return_sequences=True
# seq_model.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(1291, 10)))
# # Add attention mechanism
# seq_model.add(Dense(units=1, activation='tanh'))
# seq_model.add(Flatten())
# seq_model.add(Activation('softmax'))
# seq_model.add(RepeatVector(128))
# seq_model.add(Permute([2, 1]))
# seq_model.add(Multiply())
# seq_model.add(Lambda(lambda x: K.sum(x, axis=1)))
# # Add output layer
# seq_model.add(Dense(1, activation='sigmoid'))
# seq_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# seq_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)
# print('Sequential model performance:')
# #NOTE: We can add an embedding layer in another implementation
# loss, accuracy = seq_model.evaluate(X_test,y_test)

created


/Users/pratyushpradhan/Developer/Personal/Projects/Crypto/CryptoDataProcessingAndML/Version2/DataProcessing.py:34: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  MACD = ta.macd(df['close'])


AttributeError: 'ETL' object has no attribute 'create_tensor_with_sequence'

In [4]:
temp1 = ['1','2']
temp2 = ['2','4']
for val in temp1:
    if(val in temp2):
        print(val)

2


In [3]:
#code to convert to 3d vector
# Define sequence length
sequence_length = 14
# Create a copy of the original data
data_seq = X.copy()
# Create new columns for each day of historical data
for i in range(1, sequence_length + 1):
    for col in X.columns:
        data_seq[f'{col}_lag{i}'] = X[col].shift(i)

# Drop rows with missing values
data_seq.dropna(inplace=True)

,open,high,low,close,base_volume,ti_stoch_kd,ti_MACD,open_lag1,high_lag1,low_lag1,...,base_volume_lag13,ti_stoch_kd_lag13,ti_MACD_lag13,open_lag14,high_lag14,low_lag14,close_lag14,base_volume_lag14,ti_stoch_kd_lag14,ti_MACD_lag14
count,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,...,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000,2004.000000
mean,0.217258,0.222403,0.212381,0.217385,0.123336,0.058882,0.000000,0.217123,0.222270,0.212250,...,0.122619,0.057884,0.000000,0.215338,0.220471,0.210446,0.215464,0.122589,0.057884,0.000000
std,0.246188,0.250897,0.241633,0.246180,0.108500,0.737367,0.282631,0.246199,0.250910,0.241645,...,0.108911,0.736769,0.282631,0.246410,0.251131,0.241836,0.246393,0.108936,0.736769,0.282631
min,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,0.028409,0.029260,0.027679,0.028410,0.050881,0.000000,0.000000,0.028409,0.029260,0.027679,...,0.049663,0.000000,0.000000,0.028409,0.029260,0.027665,0.028410,0.049636,0.000000,0.000000
50%,0.085360,0.089525,0.082821,0.086042,0.096814,0.000000,0.000000,0.085159,0.089275,0.082745,...,0.096182,0.000000,0.000000,0.082228,0.084039,0.080959,0.082363,0.096182,0.000000,0.000000
75%,0.336886,0.343068,0.334052,0.336885,0.164224,1.000000,0.000000,0.336886,0.343068,0.334052,...,0.163941,1.000000,0.000000,0.334679,0.342124,0.329251,0.334650,0.163941,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
display(df.head(10))
# train test split
#note we have 2 labels, one for daily(B) another for shortterm (A) => shorterm>daily
#taking daily B 'result_B'
col_label='result_B'
training_size = int(len(df)*70)
test_size = len(df)-training_size
X_train, X_test, y_train, y_test= df[0:training_size,]


2018

In [20]:
display(X_test.head(20))

,open,high,low,close,base_volume,quote_volume,num_trades,ti_stoch_kd,ti_MACD,unix_start_datetime
1647,0.034035,0.033703,0.030982,0.030545,0.074875,0.007061,0.021772,1,0,1537747200000
1648,0.033288,0.034441,0.033507,0.034025,0.050224,0.004878,0.015403,0,0,1537660800000
1649,0.034698,0.035570,0.032579,0.033286,0.090661,0.008780,0.022729,-1,1,1537574400000
1650,0.029687,0.034938,0.029980,0.034708,0.180167,0.016744,0.046171,-1,0,1537488000000
1651,0.026679,0.029622,0.026720,0.029685,0.122812,0.010429,0.030893,0,-1,1537401600000
1652,0.026389,0.027146,0.024997,0.026677,0.106182,0.008833,0.022954,1,0,1537315200000
1653,0.023813,0.027278,0.024110,0.026335,0.125114,0.010177,0.029854,1,0,1537228800000
1654,0.028891,0.029270,0.023879,0.023792,0.176733,0.014837,0.040152,1,0,1537142400000
1655,0.029184,0.028957,0.027268,0.028875,0.091047,0.007909,0.023336,0,0,1537056000000
1656,0.026417,0.029776,0.027169,0.029112,0.094373,0.008272,0.022332,-1,0,1536969600000


In [ ]:
seq_model = Sequential()
# Add LSTM layer with return_sequences=True
seq_model.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(1291, 10))))
# Add attention mechanism
# seq_model.add(Dense(units=1, activation='tanh'))
# seq_model.add(Flatten())
# seq_model.add(Activation('softmax'))
# seq_model.add(RepeatVector(128))
# seq_model.add(Permute([2, 1]))
# seq_model.add(Multiply())
# seq_model.add(Lambda(lambda x: K.sum(x, axis=1)))
# Add output layer
seq_model.add(Dense(1, activation='sigmoid'))
seq_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
seq_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)
print('Sequential model performance:')
#NOTE: We can add an embedding layer in another implementation
loss, accuracy = seq_model.evaluate(X_test,y_test)

In [ ]:
#backup

import pandas as pd
import numpy as np
import pandas_ta as ta
import logging
import DataProcessing

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Flatten, TimeDistributed, RepeatVector, Permute, Multiply, Lambda,Bidirectional
from keras import backend as K

dataProcess = DataProcessing.ETL()
df = dataProcess.Load_Clean_Data('./historical_data/Binance_ETHUSDT_d.csv','1d')

cols_ti= ['ti_stoch_kd','ti_MACD']
cols_to_normalize = ['open','high', 'low', 'close', 'base_volume', 'quote_volume', 'num_trades']

df = dataProcess.Add_TI_Data(df, cols=cols_ti)
df = dataProcess.Add_Label(df,cols_to_normalize=cols_to_normalize)

#Note we are not taking time data (unix_start_datetime)
x_cols = cols_to_normalize+cols_ti
labels = ['result_A','result_B']

X= df[x_cols]
#note we have 2 labels, one for daily(B) another for shortterm (A) => shorterm>daily
y1= df['result_A']
y2= df['result_B']


y = y2.replace({'up': 1, 'down': -1, 'flat':0})
print(y)
y = to_categorical(y)
# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# further split training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

# print the shape of each set
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)


seq_model = Sequential()
# Add LSTM layer with return_sequences=True
seq_model.add(Bidirectional(LSTM(units=128, return_sequences=True, input_shape=(1291, 10))))
# Add attention mechanism
seq_model.add(Dense(units=1, activation='tanh'))
seq_model.add(Flatten())
seq_model.add(Activation('softmax'))
seq_model.add(RepeatVector(128))
seq_model.add(Permute([2, 1]))
seq_model.add(Multiply())
seq_model.add(Lambda(lambda x: K.sum(x, axis=1)))
# Add output layer
seq_model.add(Dense(3, activation='softmax'))
seq_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
seq_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)
print('Sequential model performance:')
#NOTE: We can add an embedding layer in another implementation
loss, accuracy = seq_model.evaluate(X_test,y_test)

In [ ]:
#code to use softmax as the last layer for classifying up down flat along with the probability

import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Create a simple neural network model
model = Sequential()
model.add(Dense(3, input_dim=4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Generate some input data
X = np.array([[1, 2, 3, 4]])

# Get the predicted probabilities for each class
y_pred = model.predict(X)

# Get the predicted class label and associated probability
predicted_class_label = np.argmax(y_pred)
predicted_probability = y_pred[0][predicted_class_label]

# Print the results
print('Predicted class label:', predicted_class_label)
print('Associated probability:', predicted_probability)

In [ ]:
#code to convert to 3d vector

import numpy as np

# Define sequence length
sequence_length = 30

# Create a copy of the original data
data_seq = stock_data.copy()

# Create new columns for each day of historical data
for i in range(1, sequence_length + 1):
    for col in stock_data.columns:
        data_seq[f'{col}_lag{i}'] = stock_data[col].shift(i)

# Drop rows with missing values
data_seq.dropna(inplace=True)

# Get X and y
X = data_seq.drop(['target_column'], axis=1)
y = data_seq['target_column']